In [49]:
# typing
from typing import List

# import gtsam
import gtsam
from gtsam import *

# import gtbook
import gtbook
from gtbook.discrete import Variables

# import local package
import gtsam_planner
from gtsam_planner import *

ModuleNotFoundError: No module named 'gtsam_planner'

In [42]:
class SASToGTSAM():
    def __init__(self, sas):
        self.sas = sas
        self.init = self.sas.initial_state
        self.goal = self.sas.goal
        self.vars = self.sas.variables
        self.ops = self.sas.operators
        self.mutex_groups = self.sas.mutex_group
        self.ops_names = []
        for op in self.ops:
            self.ops_names.append(op.name)
        self.state_keys = list(self.vars.keys())
        self.variables = Variables()

    def generate_state(self, timestep: int) -> List:
        state = []
        for var, val in self.vars.items():
            state_var = self.variables.discrete(str(var)+"_"+str(timestep), val)
            state.append(state_var)
        return state
    
    def generate_operator_key(self, timestep: int) -> gtsam.DiscreteKey:
        op_var = self.variables.discrete("op_"+str(timestep), self.ops_names)
        return op_var
    
    def generate_initial_factor(self, initial_state):
        keys = gtsam.DiscreteKeys()
        for key in initial_state:
            keys.push_back(key)
        init_values = list(self.init.values())
        init_f = gtsam_planner.MultiValueConstraint(keys, init_values)
        return init_f

    def generate_goal_factor(self, goal_state):
        keys = gtsam.DiscreteKeys()
        vals = []
        for goal_var, goal_val in self.goal.items():
            keys.push_back(goal_state[self.state_keys.index(goal_var)])
            vals.append(goal_val)
        goal_f = gtsam_planner.MultiValueConstraint(keys, vals)
        return goal_f
    
    def generate_op_null(self, state_t, state_tp, operator):
        vals = []
        op_keys = set()
        multi_keys = gtsam.DiscreteKeys()
        for pre_var, pre_val in operator.precondition.items():
            key = state_t[self.state_keys.index(pre_var)]
            op_keys.add(key)
            if pre_val == -1:
                continue
            multi_keys.push_back(key)
            vals.append(pre_val)
        
        for eff_var, eff_val in operator.effect.items():
            key = state_tp[self.state_keys.index(eff_var)]
            op_keys.add(key)
            if eff_val == -1:
                continue
            multi_keys.push_back(key)
            vals.append(eff_val)
        
        if operator.num_prevail > 0:
            for var, val in operator.prevail.items():
                key_t = state_t[self.state_keys.index(var)]
                key_tp = state_tp[self.state_keys.index(var)]
                op_keys.add(key_t)
                op_keys.add(key_tp)
                multi_keys.push_back(key_t)
                multi_keys.push_back(key_tp)
                vals.extend([val,val])

        assert len(op_keys) % 2 == 0
        frame_keys = set()
        null_keys = gtsam.DiscreteKeys()
        for var in state_t+state_tp:
            if var not in op_keys:
                null_keys.push_back(var)
                frame_keys.add(var)

        op_f = gtsam_planner.MultiValueConstraint(multi_keys, vals)
        null_f = gtsam_planner.NullConstraint(null_keys)
        return op_f, null_f, op_keys, frame_keys

    def generate_mutex_factor(self, state_t):
        state = list(self.vars.keys())
        mutex_variables = []
        mutex_values = []
        
        for mutex_group in self.mutex_groups:
            var_group = []
            val_group = []
            for var, val in mutex_group:
                state_var = state_t[state.index(var)]
                var_group.append(state_var)
                val_group.append(val)
            mutex_variables.append(var_group)
            mutex_values.append(val_group)
        
        factors = []
        for mutex_var, mutex_val in zip(mutex_variables, mutex_values):
            keys = gtsam.DiscreteKeys()
            for var in mutex_var:
                keys.push_back(var)
            mutex = gtsam_planner.MutexConstraint(keys, mutex_val)
            factors.append(mutex)
        return factors
    
    def generate_frame_op_factor(self, state_t, state_tp, op_key):
        op_consts = []
        null_consts = []
        
        op_keys_set = set()
        op_keys_set.add(op_key)
        
        frame_keys_set = set()
        frame_keys_set.add(op_key)
        
        for op in self.ops:
            op_const, null_const, op_keys, frame_keys = self.generate_op_null(state_t, state_tp, op)
            op_consts.append(op_const)
            null_consts.append(null_const)
            op_keys_set = op_keys_set.union(op_keys)
            frame_keys_set = frame_keys_set.union(frame_keys)

        op_dkeys = gtsam.DiscreteKeys()
        for key in op_keys_set:
            op_dkeys.push_back(key)
        
        frame_dkeys = gtsam.DiscreteKeys()
        for key in frame_keys_set:
            frame_dkeys.push_back(key)

        op_factor = gtsam_planner.OperatorOrConstraint(op_key, op_dkeys, op_consts)
        frame_factor = gtsam_planner.FrameConstraint(op_key, frame_dkeys, null_consts)
        return op_factor, frame_factor

In [45]:
init = sas.initial_state
goal = sas.goal
vars = sas.variables
ops = sas.operators
ops_names = []
for op in ops:
    ops_names.append(op.name)
mutex_groups = sas.mutex_group
variables = Variables()
timestep = 0

In [46]:
# generate state
state = []
for var, val in vars.items():
    state_var = variables.discrete(str(var)+"_"+str(timestep), val)
    state.append(state_var)
print(state)

[(0, 5), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 5), (7, 5), (8, 5)]


In [47]:
# generate operator key
op_var = variables.discrete("op_"+str(timestep), ops_names)
print(op_var)

(9, 32)


In [48]:
init

{0: 4, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 4, 7: 4, 8: 4}

In [ ]:
# generate initial factor
keys = gtsam.DiscreteKeys()
for key in initial_state:
    keys.push_back(key)
init_values = list(self.init.values())
init_f = gtsam_planner.MultiValueConstraint(keys, init_values)
return init_f

In [13]:
ops

In [45]:
l = [1, 2, 3, 4, 5, 6, 7, 8]
v0 = (0, 1)
v1 = (0, 1)
v2 = (0, 1)
vars = [v0, v1, v2]
total = 24
layer0 = len(l)/len(v0)
layer1 = layer0/len(v1)
layer2 = layer1/len(v2)

In [46]:
layer0

4.0

In [47]:
layer1

2.0

In [27]:
input = []
for var in vars:
    input.append(list(range(len(var))))
    prods = list(product(*input))


In [28]:
prods

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [31]:
# 0: total/len(v0)
# v0 = 0 at 0 ~ 11
# v0 = 1 at 12 ~ 23

[1, 2, 3, 4, 5, 6, 7, 8]

In [40]:
chunks = [l[x:len(l)//len(v0)] for x in range(0, len(l), len(l)//len(v0))]

In [41]:
chunks

[[1, 2, 3, 4], []]

In [48]:
class Tree:
    def __init__(self, data):
        self.children = []
        self.data = data

In [56]:
layer0 = Tree(v0)
layer1 = Tree(v1)
layer2 = Tree(v2)
layer0.children = [layer1, layer1]
layer1.children = [layer2, layer2]

In [58]:
    def search(self, node, data):
        """
        Search function will search a node into tree.
        """
        # if root is None or root is the search data.
        if node is None or node.data == data:
            return node

        if node.data < data:
            return self.search(node.right, data)
        else:
            return self.search(node.left, data)

In [ ]:
# v1 = 0 at 0 ~ 5
# v1 = 1 at 6 ~ 11

In [3]:
def bfs(graph, start_node):
    visit = list()
    queue = list()

    queue.append(start_node)

    while queue:
        node = queue.pop(0)
        if node not in visit:
            visit.append(node)
            queue.extend(graph[node])

    return visit

In [12]:
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['B', 'B'],
    'E': ['B', 'B'],
    'F': ['C', 'C'],
    'G': ['C', 'C']
}


In [13]:
bfs(graph, 'A')

['A', 'B', 'C', 'D', 'E', 'F', 'G']